In [1]:
#!/usr/bin/env python
import re, random, math, collections, itertools

PRINT_ERRORS=0

#------------- Function Definitions ---------------------


def readFiles(sentimentDictionary,sentencesTrain,sentencesTest,sentencesNokia):

    #reading pre-labeled input and splitting into lines
    posSentences = open('rt-polarity.pos', 'r', encoding="ISO-8859-1")
    posSentences = re.split(r'\n', posSentences.read())

    negSentences = open('rt-polarity.neg', 'r', encoding="ISO-8859-1")
    negSentences = re.split(r'\n', negSentences.read())

    posSentencesNokia = open('nokia-pos.txt', 'r')
    posSentencesNokia = re.split(r'\n', posSentencesNokia.read())

    negSentencesNokia = open('nokia-neg.txt', 'r', encoding="ISO-8859-1")
    negSentencesNokia = re.split(r'\n', negSentencesNokia.read())

    posDictionary = open('positive-words.txt', 'r', encoding="ISO-8859-1")
    posWordList = posDictionary.readlines()
    posWordList = [line.strip() for line in posWordList if not line.startswith(";") and not line == '\n']
    #posWordList = re.findall(r"[a-z\-]+", posDictionary.read())

    negDictionary = open('negative-words.txt', 'r', encoding="ISO-8859-1")
    negWordList = negDictionary.readlines()
    negWordList = [line.strip() for line in negWordList if not line.startswith(";") and not line == '\n']
    #negWordList = re.findall(r"[a-z\-]+", negDictionary.read())

    for i in posWordList:
        sentimentDictionary[i] = 1
    for i in negWordList:
        sentimentDictionary[i] = -1

    #create Training and Test Datsets:
    #We want to test on sentences we haven't trained on, to see how well the model generalses to previously unseen sentences

  #create 90-10 split of training and test data from movie reviews, with sentiment labels    
    for i in posSentences:
        if random.randint(1,10)<2:
            sentencesTest[i]="positive"
        else:
            sentencesTrain[i]="positive"

    for i in negSentences:
        if random.randint(1,10)<2:
            sentencesTest[i]="negative"
        else:
            sentencesTrain[i]="negative"

    #create Nokia Datset:
    for i in posSentencesNokia:
            sentencesNokia[i]="positive"
    for i in negSentencesNokia:
            sentencesNokia[i]="negative"

#----------------------------End of data initialisation ----------------#

#calculates p(W|Positive), p(W|Negative) and p(W) for all words in training data
def trainBayes(sentencesTrain, pWordPos, pWordNeg, pWord):
    posFeatures = [] # [] initialises a list [array]
    negFeatures = [] 
    freqPositive = {} # {} initialises a dictionary [hash function]
    freqNegative = {}
    dictionary = {}
    posWordsTot = 0
    negWordsTot = 0
    allWordsTot = 0

    #iterate through each sentence/sentiment pair in the training data
    for sentence, sentiment in sentencesTrain.items():
        wordList = re.findall(r"[\w']+", sentence)

        #TO DO:
        #Populate bigramList (initialised below) by concatenating adjacent words in the sentence.
        #You might want to seperate the words by _ for readability, so bigrams such as:
        #You_might, might_want, want_to, to_seperate.... 

        bigramList=wordList.copy() #initialise bigramList
        for x in range(len(wordList)-1):
           bigramList.append(wordList[x]+"_" + wordList[x+1])
        
        trigramList=bigramList.copy() #initialise bigramList
        for x in range(len(wordList)-2):
            trigramList.append(wordList[x]+"" + wordList[x+1]+"" + wordList[x+2])


 
        #-------------Finish populating bigramList ------------------#
        
        #TO DO: when you have populated bigramList, uncomment out the line below and , and comment out the unigram line to make use of bigramList rather than wordList
        
        for word in  trigramList: #calculate over bigrams
        # for word in wordList: #calculate over unigrams
            allWordsTot += 1 # keeps count of total words in dataset
            if not (word in dictionary):
                dictionary[word] = 1
            if sentiment=="positive" :
                posWordsTot += 1 # keeps count of total words in positive class

                #keep count of each word in positive context
                if not (word in freqPositive):
                    freqPositive[word] = 1
                else:
                    freqPositive[word] += 1    
            else:
                negWordsTot+=1# keeps count of total words in negative class
                
                #keep count of each word in positive context
                if not (word in freqNegative):
                    freqNegative[word] = 1
                else:
                    freqNegative[word] += 1

    for word in dictionary:
        #do some smoothing so that minimum count of a word is 1
        if not (word in freqNegative):
            freqNegative[word] = 1
        if not (word in freqPositive):
            freqPositive[word] = 1

        # Calculate p(word|positive)
        pWordPos[word] = freqPositive[word] / float(posWordsTot)

        # Calculate p(word|negative) 
        pWordNeg[word] = freqNegative[word] / float(negWordsTot)

        # Calculate p(word)
        pWord[word] = (freqPositive[word] + freqNegative[word]) / float(allWordsTot) 

#---------------------------End Training ----------------------------------

#implement naive bayes algorithm
#INPUTS:
#  sentencesTest is a dictonary with sentences associated with sentiment 
#  dataName is a string (used only for printing output)
#  pWordPos is dictionary storing p(word|positive) for each word
#     i.e., pWordPos["apple"] will return a real value for p("apple"|positive)
#  pWordNeg is dictionary storing p(word|negative) for each word
#  pWord is dictionary storing p(word)
#  pPos is a real number containing the fraction of positive reviews in the dataset
def testBayes(sentencesTest, dataName, pWordPos, pWordNeg, pWord,pPos):
    pNeg=1-pPos

    #These variables will store results (you do not need them)
    total=0
    correct=0
    totalpos=0
    totalpospred=0
    totalneg=0
    totalnegpred=0
    correctpos=0
    correctneg=0

    #for each sentence, sentiment pair in the dataset
    for sentence, sentiment in sentencesTest.items():
        wordList = re.findall(r"[\w']+", sentence)#collect all words

        
        #TO DO: Exactly what you did in the training function:
        #Populate bigramList by concatenating adjacent words in wordList.

        bigramList=wordList.copy() #initialise bigramList
        for x in range(len(wordList)-1):
            bigramList.append(wordList[x]+"_" + wordList[x+1])
        trigramList=bigramList.copy() #initialise bigramList
        for x in range(len(wordList)-2):
            trigramList.append(wordList[x]+"" + wordList[x+1]+"" + wordList[x+2])
        
        

#------------------finished populating bigramList--------------
        pPosW=pPos
        pNegW=pNeg

        for word in  trigramList: #calculate over bigrams
#        for word in wordList: #calculate over unigrams
            if word in pWord:
                if pWord[word]>0.00000001:
                    pPosW *=pWordPos[word]
                    pNegW *=pWordNeg[word]

        prob=0;            
        if pPosW+pNegW >0:
            prob=pPosW/float(pPosW+pNegW)


        total+=1
        if sentiment=="positive":
            totalpos+=1
            if prob>0.5:
                correct+=1
                correctpos+=1
                totalpospred+=1
            else:
                correct+=0
                totalnegpred+=1
                if PRINT_ERRORS:
                    print ("ERROR (pos classed as neg %0.2f):" %prob + sentence)
        else:
            totalneg+=1
            if prob<=0.5:
                correct+=1
                correctneg+=1
                totalnegpred+=1
            else:
                correct+=0
                totalpospred+=1
                if PRINT_ERRORS:
                    print ("ERROR (neg classed as pos %0.2f):" %prob + sentence)
 
    acc=correct/float(total)
    print (dataName + " Accuracy (All)=%0.2f" % acc + " (%d" % correct + "/%d" % total + ")\n")

    precision_pos=correctpos/float(totalpospred)
    recall_pos=correctpos/float(totalpos)
    precision_neg=correctneg/float(totalnegpred)
    recall_neg=correctneg/float(totalneg)
    f_pos=2*precision_pos*recall_pos/(precision_pos+recall_pos);
    f_neg=2*precision_neg*recall_neg/(precision_neg+recall_neg);

    print (dataName + " Precision (Pos)=%0.2f" % precision_pos + " (%d" % correctpos + "/%d" % totalpospred + ")")
    print (dataName + " Recall (Pos)=%0.2f" % recall_pos + " (%d" % correctpos + "/%d" % totalpos + ")")
    print (dataName + " F-measure (Pos)=%0.2f" % f_pos)

    print (dataName + " Precision (Neg)=%0.2f" % precision_neg + " (%d" % correctneg + "/%d" % totalnegpred + ")")
    print (dataName + " Recall (Neg)=%0.2f" % recall_neg + " (%d" % correctneg + "/%d" % totalneg + ")")
    print (dataName + " F-measure (Neg)=%0.2f" % f_neg + "\n")



# This is a simple classifier that uses a sentiment dictionary to classify 
# a sentence. For each word in the sentence, if the word is in the positive 
# dictionary, it adds 1, if it is in the negative dictionary, it subtracts 1. 
# If the final score is above a threshold, it classifies as "Positive", 
# otherwise as "Negative"
def testDictionary(sentencesTest, dataName, sentimentDictionary, threshold):
    total=0
    correct=0
    totalpos=0
    totalneg=0
    totalpospred=0
    totalnegpred=0
    correctpos=0
    correctneg=0
    for sentence, sentiment in sentencesTest.items():
        Words = re.findall(r"[\w']+", sentence)
        score=0
        for word in Words:
            if word in sentimentDictionary:
               score+=sentimentDictionary[word]
 
        total+=1
        if sentiment=="positive":
            totalpos+=1
            if score>=threshold:
                correct+=1
                correctpos+=1
                totalpospred+=1
            else:
                correct+=0
                totalnegpred+=1
        else:
            totalneg+=1
            if score<threshold:
                correct+=1
                correctneg+=1
                totalnegpred+=1
            else:
                correct+=0
                totalpospred+=1
 
    acc=correct/float(total)
    print (dataName + " Accuracy (All)=%0.2f" % acc + " (%d" % correct + "/%d" % total + ")\n")
    precision_pos=correctpos/float(totalpospred)
    recall_pos=correctpos/float(totalpos)
    precision_neg=correctneg/float(totalnegpred)
    recall_neg=correctneg/float(totalneg)
    f_pos=2*precision_pos*recall_pos/(precision_pos+recall_pos);
    f_neg=2*precision_neg*recall_neg/(precision_neg+recall_neg);


    print (dataName + " Precision (Pos)=%0.2f" % precision_pos + " (%d" % correctpos + "/%d" % totalpospred + ")")
    print (dataName + " Recall (Pos)=%0.2f" % recall_pos + " (%d" % correctpos + "/%d" % totalpos + ")")
    print (dataName + " F-measure (Pos)=%0.2f" % f_pos)

    print (dataName + " Precision (Neg)=%0.2f" % precision_neg + " (%d" % correctneg + "/%d" % totalnegpred + ")")
    print (dataName + " Recall (Neg)=%0.2f" % recall_neg + " (%d" % correctneg + "/%d" % totalneg + ")")
    print (dataName + " F-measure (Neg)=%0.2f" % f_neg + "\n")

#-----------------------------------------------------------------------------------------



#Print out n most useful predictors
def mostUseful(pWordPos, pWordNeg, pWord, n):
    predictPower={}
    for word in pWord:
        if pWordNeg[word]<0.0000001:
            predictPower[word]=1000000000
        else:
            predictPower[word]=pWordPos[word] / (pWordPos[word] + pWordNeg[word])
            

    sortedPower = sorted(predictPower, key=predictPower.get)
    head, tail = sortedPower[:n], sortedPower[len(predictPower)-n:]
    print ("NEGATIVE:")
    print (head)
    print ("\nPOSITIVE:")
    print (tail)




#---------- Main Script --------------------------


sentimentDictionary={} # {} initialises a dictionary [hash function]
sentencesTrain={}
sentencesTest={}
sentencesNokia={}

#initialise datasets and dictionaries
readFiles(sentimentDictionary,sentencesTrain,sentencesTest,sentencesNokia)

pWordPos={} # p(W|Positive)
pWordNeg={} # p(W|Negative)
pWord={}    # p(W) 

#build conditional probabilities using training data
trainBayes(sentencesTrain, pWordPos, pWordNeg, pWord)

#run naive bayes classifier on datasets
#print ("Naive Bayes")
testBayes(sentencesTrain,  "Films (Train Data, Naive Bayes)\t", pWordPos, pWordNeg, pWord,0.5)
testBayes(sentencesTest,  "Films  (Test Data, Naive Bayes)\t", pWordPos, pWordNeg, pWord,0.5)
testBayes(sentencesNokia, "Nokia   (All Data,  Naive Bayes)\t", pWordPos, pWordNeg, pWord,0.7)



#run sentiment dictionary based classifier on datasets
#testDictionary(sentencesTrain,  "Films (Train Data, Rule-Based)\t", sentimentDictionary, -4)
#testDictionary(sentencesTest,  "Films  (Test Data, Rule-Based)\t",  sentimentDictionary, 0)
#testDictionary(sentencesNokia, "Nokia   (All Data, Rule-Based)\t",  sentimentDictionary, 0)


# print most useful words
#mostUseful(pWordPos, pWordNeg, pWord, 50)





Films (Train Data, Naive Bayes)	 Accuracy (All)=0.82 (7849/9598)

Films (Train Data, Naive Bayes)	 Precision (Pos)=0.93 (3288/3520)
Films (Train Data, Naive Bayes)	 Recall (Pos)=0.68 (3288/4805)
Films (Train Data, Naive Bayes)	 F-measure (Pos)=0.79
Films (Train Data, Naive Bayes)	 Precision (Neg)=0.75 (4561/6078)
Films (Train Data, Naive Bayes)	 Recall (Neg)=0.95 (4561/4793)
Films (Train Data, Naive Bayes)	 F-measure (Neg)=0.84

Films  (Test Data, Naive Bayes)	 Accuracy (All)=0.77 (825/1065)

Films  (Test Data, Naive Bayes)	 Precision (Pos)=0.78 (394/502)
Films  (Test Data, Naive Bayes)	 Recall (Pos)=0.75 (394/526)
Films  (Test Data, Naive Bayes)	 F-measure (Pos)=0.77
Films  (Test Data, Naive Bayes)	 Precision (Neg)=0.77 (431/563)
Films  (Test Data, Naive Bayes)	 Recall (Neg)=0.80 (431/539)
Films  (Test Data, Naive Bayes)	 F-measure (Neg)=0.78

Nokia   (All Data,  Naive Bayes)	 Accuracy (All)=0.62 (164/266)

Nokia   (All Data,  Naive Bayes)	 Precision (Pos)=0.81 (109/134)
Nokia   (All 

### Adding new rules

In [2]:
def testDictionary_new_rules(sentencesTest, dataName, sentimentDictionary, threshold, score_count=2):
    total=0
    correct=0
    totalpos=0
    totalneg=0
    totalpospred=0
    totalnegpred=0
    correctpos=0
    correctneg=0
    count=0
    lst = []
    for sentence, sentiment in sentencesTest.items():
        Words = re.findall(r"[\w']+", sentence)
        score=0
        left_split = []
        not_list = []
        for word in Words:
            if 'but' in Words:
                for i in range(len(Words) - 1):
                    left_split.append(Words[i])
                    if Words[i + 1] == 'but':
                        break
            
            elif 'not bad' in Words:
                for i in range(len(Words) - 1):
                    left_split.append(Words[i])
                    if Words[i + 1] == 'not bad':
                        break
                        
            elif 'only' in Words:
                for i in range(len(Words) - 1):
                    left_split.append(Words[i])
                    if Words[i + 1] == 'only':
                        break

            
                #print(left_split)
        
                if len(left_split) > 0:
                    for word in left_split:
                        if word in sentimentDictionary:
                            if sentimentDictionary[word] == -1:
                                score += score_count
                            elif sentimentDictionary[word] == 1:
                                score -= score_count
                            else:
                                score +=0
                            count+=1
                            #print(score)     
            elif word in sentimentDictionary:
                score+=sentimentDictionary[word]
                
        #print(sentence, sentiment)
        total+=1
        if sentiment=="positive":
            totalpos+=1
            if score>=threshold:
                correct+=1
                correctpos+=1
                totalpospred+=1
            else:
                correct+=0
                totalnegpred+=1
        else:
            totalneg+=1
            if score<threshold:
                correct+=1
                correctneg+=1
                totalnegpred+=1
            else:
                correct+=0
                totalpospred+=1

    acc=correct/float(total)
    print (dataName + " Accuracy (All)=%0.2f" % acc + " (%d" % correct + "/%d" % total + ")\n")
    precision_pos=correctpos/float(totalpospred)
    recall_pos=correctpos/float(totalpos)
    precision_neg=correctneg/float(totalnegpred) if (totalnegpred) else 0
    recall_neg=correctneg/float(totalneg) #if (totalneg) else 0
    f_pos=2*precision_pos*recall_pos/(precision_pos+recall_pos);
    f_neg=2*precision_neg*recall_neg/(precision_neg+recall_neg) if (precision_neg+recall_neg) else 0;


    print (dataName + " Precision (Pos)=%0.2f" % precision_pos + " (%d" % correctpos + "/%d" % totalpospred + ")")
    print (dataName + " Recall (Pos)=%0.2f" % recall_pos + " (%d" % correctpos + "/%d" % totalpos + ")")
    print (dataName + " F-measure (Pos)=%0.2f" % f_pos)

    print (dataName + " Precision (Neg)=%0.2f" % precision_neg + " (%d" % correctneg + "/%d" % totalnegpred + ")")
    print (dataName + " Recall (Neg)=%0.2f" % recall_neg + " (%d" % correctneg + "/%d" % totalneg + ")")
    print (dataName + " F-measure (Neg)=%0.2f" % f_neg + "\n")
    


In [3]:
testDictionary_new_rules(sentencesTrain,  "Films (Train Data, Rule-Based)\t", sentimentDictionary, 1)
testDictionary_new_rules(sentencesTest,  "Films  (Test Data, Rule-Based)\t",  sentimentDictionary, 1)
testDictionary_new_rules(sentencesNokia, "Nokia   (All Data, Rule-Based)\t",  sentimentDictionary, 1)

Films (Train Data, Rule-Based)	 Accuracy (All)=0.64 (6122/9598)

Films (Train Data, Rule-Based)	 Precision (Pos)=0.70 (2316/3303)
Films (Train Data, Rule-Based)	 Recall (Pos)=0.48 (2316/4805)
Films (Train Data, Rule-Based)	 F-measure (Pos)=0.57
Films (Train Data, Rule-Based)	 Precision (Neg)=0.60 (3806/6295)
Films (Train Data, Rule-Based)	 Recall (Neg)=0.79 (3806/4793)
Films (Train Data, Rule-Based)	 F-measure (Neg)=0.69

Films  (Test Data, Rule-Based)	 Accuracy (All)=0.65 (694/1065)

Films  (Test Data, Rule-Based)	 Precision (Pos)=0.72 (254/353)
Films  (Test Data, Rule-Based)	 Recall (Pos)=0.48 (254/526)
Films  (Test Data, Rule-Based)	 F-measure (Pos)=0.58
Films  (Test Data, Rule-Based)	 Precision (Neg)=0.62 (440/712)
Films  (Test Data, Rule-Based)	 Recall (Neg)=0.82 (440/539)
Films  (Test Data, Rule-Based)	 F-measure (Neg)=0.70

Nokia   (All Data, Rule-Based)	 Accuracy (All)=0.80 (212/266)

Nokia   (All Data, Rule-Based)	 Precision (Pos)=0.92 (145/158)
Nokia   (All Data, Rule-Based)	

### Adding trigram 

In [4]:
def testDictionary_tri(sentencesTest, dataName, sentimentDictionary, threshold):
    total=0
    correct=0
    totalpos=0
    totalneg=0
    totalpospred=0
    totalnegpred=0
    correctpos=0
    correctneg=0
    for sentence, sentiment in sentencesTest.items():
        wordList = re.findall(r"[\w']+", sentence)
        trigramList = wordList.copy()  # initialise trigramList
        for x in range(len(wordList) - 2):
              trigramList.append(wordList[x] + "_" + wordList[x + 1] + "_" + wordList[x + 2])
        score=0
        for trigram in trigramList:
            Words = trigram.split("_")
            for word in Words:
                if word in sentimentDictionary:
                   score+=sentimentDictionary[word]
            total+=1
            if sentiment=="positive":
                totalpos+=1
                if score>=threshold:
                    correct+=1
                    correctpos+=1
                    totalpospred+=1
                else:
                    correct+=0
                    totalnegpred+=1
            else:
                totalneg+=1
                if score<threshold:
                    correct+=1
                    correctneg+=1
                    totalnegpred+=1
                else:
                    correct+=0
                    totalpospred+=1
 
    acc=correct/float(total)
    print (dataName + " Accuracy (All)=%0.2f" % acc + " (%d" % correct + "/%d" % total + ")\n")
    precision_pos=correctpos/float(totalpospred)
    recall_pos=correctpos/float(totalpos)
    precision_neg=correctneg/float(totalnegpred)
    recall_neg=correctneg/float(totalneg)
    f_pos=2*precision_pos*recall_pos/(precision_pos+recall_pos);
    f_neg=2*precision_neg*recall_neg/(precision_neg+recall_neg);


    print (dataName + " Precision (Pos)=%0.2f" % precision_pos + " (%d" % correctpos + "/%d" % totalpospred + ")")
    print (dataName + " Recall (Pos)=%0.2f" % recall_pos + " (%d" % correctpos + "/%d" % totalpos + ")")
    print (dataName + " F-measure (Pos)=%0.2f" % f_pos)

    print (dataName + " Precision (Neg)=%0.2f" % precision_neg + " (%d" % correctneg + "/%d" % totalnegpred + ")")
    print (dataName + " Recall (Neg)=%0.2f" % recall_neg + " (%d" % correctneg + "/%d" % totalneg + ")")
    print (dataName + " F-measure (Neg)=%0.2f" % f_neg + "\n")

In [5]:
testDictionary_tri(sentencesTrain,  "Films (Train Data, Rule-Based)\t", sentimentDictionary, 1)
testDictionary_tri(sentencesTest,  "Films  (Test Data, Rule-Based)\t",  sentimentDictionary, 1)
testDictionary_tri(sentencesNokia, "Nokia   (All Data, Rule-Based)\t",  sentimentDictionary, 1)

Films (Train Data, Rule-Based)	 Accuracy (All)=0.61 (210848/344308)

Films (Train Data, Rule-Based)	 Precision (Pos)=0.64 (88692/138049)
Films (Train Data, Rule-Based)	 Recall (Pos)=0.51 (88692/172795)
Films (Train Data, Rule-Based)	 F-measure (Pos)=0.57
Films (Train Data, Rule-Based)	 Precision (Neg)=0.59 (122156/206259)
Films (Train Data, Rule-Based)	 Recall (Neg)=0.71 (122156/171513)
Films (Train Data, Rule-Based)	 F-measure (Neg)=0.65

Films  (Test Data, Rule-Based)	 Accuracy (All)=0.61 (22657/37138)

Films  (Test Data, Rule-Based)	 Precision (Pos)=0.64 (8916/14025)
Films  (Test Data, Rule-Based)	 Recall (Pos)=0.49 (8916/18288)
Films  (Test Data, Rule-Based)	 F-measure (Pos)=0.55
Films  (Test Data, Rule-Based)	 Precision (Neg)=0.59 (13741/23113)
Films  (Test Data, Rule-Based)	 Recall (Neg)=0.73 (13741/18850)
Films  (Test Data, Rule-Based)	 F-measure (Neg)=0.65

Nokia   (All Data, Rule-Based)	 Accuracy (All)=0.70 (5457/7827)

Nokia   (All Data, Rule-Based)	 Precision (Pos)=0.85 (332

### Checking most used words

In [6]:
mostUseful(pWordPos, pWordNeg, pWord, 20)

NEGATIVE:
['unfunny', 'supposed', 'generic', 'mediocre', 'badly', 'routine', 'supposed_to', 'the_problem', 'waste', 'mindless', 'poorly', 'has_no', '90_minutes', 'ends_up', 'pointless', 'boring', 'stale', 'disguise', 'too_bad', 'only_thing']

POSITIVE:
['an_engaging', 'gem', 'portraitofa', 'wonderful', 'chilling', 'love_and', 'refreshingly', 'performances_from', 'in_years', 'riveting', 'refreshing', 'mesmerizing', 'realistic', 'a_smart', 'unique', 'what_makes', 'lookatthe', 'inventive', 'engrossing', 'ofthebest']


### checking how many words used from sentiment dictonary to dictonary based approch

In [7]:
def head_tail(pWordPos, pWordNeg, pWord, n):
    predictPower={}
    for word in pWord:
        if pWordNeg[word]<0.0000001:
            predictPower[word]=1000000000
        else:
            predictPower[word]=pWordPos[word] / (pWordPos[word] + pWordNeg[word])


    sortedPower = sorted(predictPower, key=predictPower.get)
    head, tail = sortedPower[:n], sortedPower[len(predictPower)-n:]
    
    return head, tail

In [8]:
pcnt=0
ncnt=0
head, tail = head_tail(pWordPos, pWordNeg, pWord, 100)
print('Negative Words in SentimentDictionary are')
print('')
for i in range(len(head)):
    if head[i] in sentimentDictionary:
        ncnt+=1
        print(head[i])
    else:
        pass
print(' ')
print('Positive Words in SentimentDictionary are')
print('')
for i in range(len(tail)):
    if tail[i] in sentimentDictionary:
        pcnt+=1
        print(tail[i])
    else:
        pass

print('total positive words in sentimentDictionary are:',pcnt,' total  words',len(tail))
print('total negative words in sentimentDictionary are:',ncnt,' total  words',len(head))


Negative Words in SentimentDictionary are

mediocre
badly
waste
mindless
poorly
pointless
boring
stale
bore
stupid
tiresome
dull
clumsy
inept
offensive
lousy
junk
horrible
annoying
pathetic
sadly
incoherent
banal
missed
crap
stunt
 
Positive Words in SentimentDictionary are

heartbreaking
hopeful
smarter
sadness
powerful
warm
evocative
captivating
unexpected
timely
polished
respect
heartwarming
subversive
jealousy
dazzling
wonderfully
beauty
playful
lively
extraordinary
tender
gem
wonderful
refreshing
mesmerizing
realistic
inventive
engrossing
total positive words in sentimentDictionary are: 29  total  words 100
total negative words in sentimentDictionary are: 26  total  words 100
